In [2]:
import sys

In [3]:
!cp -r /kaggle/input/mast3r-fix/mast3r-bin /kaggle/working
!cp -r /kaggle/input/mast3r-fix/mast3r /kaggle/working

In [11]:
!cp -r /kaggle/input/mast3r-fix/mast3r-wheels /kaggle/working

In [12]:
!pip install torch torchvision torchaudio --no-index --find-links=/kaggle/working/mast3r-wheels

Looking in links: /kaggle/working/mast3r-wheels


In [13]:
# 离线安装所有依赖（不联网）
!pip install --no-index --find-links=/kaggle/working/mast3r-wheels \
    -r /kaggle/working/mast3r/requirements.txt \
    -r /kaggle/working/mast3r/dust3r/requirements.txt \
    -r /kaggle/working/mast3r/dust3r/requirements_optional.txt

Looking in links: /kaggle/working/mast3r-wheels
Processing /kaggle/working/mast3r-wheels/roma-1.5.3-py3-none-any.whl (from -r /kaggle/working/mast3r/dust3r/requirements.txt (line 3))
Processing /kaggle/working/mast3r-wheels/gradio-5.33.0-py3-none-any.whl (from -r /kaggle/working/mast3r/dust3r/requirements.txt (line 4))
Processing /kaggle/working/mast3r-wheels/trimesh-4.6.11-py3-none-any.whl (from -r /kaggle/working/mast3r/dust3r/requirements.txt (line 10))
Processing /kaggle/working/mast3r-wheels/pyglet-1.5.31-py3-none-any.whl (from -r /kaggle/working/mast3r/dust3r/requirements.txt (line 12))
Processing /kaggle/working/mast3r-wheels/pillow_heif-0.22.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from -r /kaggle/working/mast3r/dust3r/requirements_optional.txt (line 1))
Processing /kaggle/working/mast3r-wheels/pyrender-0.1.45-py3-none-any.whl (from -r /kaggle/working/mast3r/dust3r/requirements_optional.txt (line 2))
Processing /kaggle/working/mast3r-wheels/kapture-1.1.10-p

In [14]:
!pip install faiss-gpu-cu12 --no-index --find-links=/kaggle/working/mast3r-wheels

Looking in links: /kaggle/working/mast3r-wheels
Processing /kaggle/working/mast3r-wheels/faiss_gpu_cu12-1.11.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [15]:
!ls /kaggle/working/mast3r/asmk/asmk

asmk_method.py	hamming.cpython-311-x86_64-linux-gnu.so  inverted_file.py
codebook.py	index.py				 io_helpers.py
functional.py	__init__.py				 kernel.py


In [16]:

# 加入离线安装依赖包路径
sys.path.insert(0, "/kaggle/working/mast3r-bin")

# 加入源码主目录（包含 mast3r, dust3r 等子目录）
sys.path.insert(0, "/kaggle/working/mast3r")

In [17]:
%cd /kaggle/working/mast3r
!PYTHONPATH="/kaggle/working/mast3r/asmk:$PYTHONPATH" python3 demo.py --model_name MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric --share

/kaggle/working/mast3r
/kaggle/working/mast3r/dust3r/dust3r/cloud_opt/base_opt.py:275: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
Traceback (most recent call last):
  File "/kaggle/working/mast3r/dust3r/dust3r/model.py", line 82, in from_pretrained
    model = super(AsymmetricCroCo3DStereo, cls).from_pretrained(pretrained_model_name_or_path, **kw)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_validators.py", line 114, in _inner_fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/hub_mixin.py", line 566, in from_pretrained
    instance = cls._from_pretrained(
               ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/hub_mixin